In [66]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
import string

from tkinter import filedialog
from tkinter import *

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [68]:
### Select CategorizedOCI.csv file from your working directory ###

root = Tk()
root.filename =  filedialog.askopenfilename()
print (root.filename)
root.destroy()

C:/Users/krish/Desktop/UMD/Capstone/Categorized dataset/OciByIndustry.csv


In [69]:
jobs=pd.read_csv(root.filename)

In [70]:
### Select the skillwords.csv from your working directory ###

root = Tk()
root.filename =  filedialog.askopenfilename()
print (root.filename)
root.destroy()

C:/Users/krish/Desktop/UMD/Capstone/skillwords.csv


In [71]:
## hard skills/tool/concept knowledge required ##
s = pd.read_csv(root.filename)
s['Skills'] = s['Skills'].str.strip()
skillset = s['Skills'].unique()

In [72]:
data = jobs['Job Description'].tolist()

In [73]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [74]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [75]:
for i in range(len(data)):
    data[i] = preprocess(data[i])

In [76]:
#### creating a set of multiworded skills and single worded skill 
skill_words = []
skill_keywords1 = []
skill_keywords2 = []
for i in skillset:
        skill_words = i.split(' ')
        if len(skill_words) > 1:
            skill_keywords2.append(i)
        else:
            skill_keywords1.append(i)

In [77]:
degree_dict = {'bs': 1, 'bachelor': 1, 'undergraduate': 1, 
               'master': 2, 'graduate': 2, 'mba': 2.5, 
               'phd': 3, 'ph.d': 3, 'ba': 1, 'ma': 2,
               'postdoctoral': 4, 'postdoc': 4, 'doctorate': 3}


degree_dict2 = {'advanced degree': 2, 'ms or': 2, 'ms degree': 2, '4 year degree': 1, 'bs/': 1, 'ba/': 1,
                '4-year degree': 1, 'b.s.': 1, 'm.s.': 2, 'm.s': 2, 'b.s': 1, 'phd/': 3, 'ph.d.': 3, 'ms/': 2,
                'm.s/': 2, 'm.s./': 2, 'msc/': 2, 'master/': 2, 'master\'s/': 2, 'bachelor\s/': 1}
degree_keywords2 = set(degree_dict2.keys())

In [79]:
import pandas as pd
df = pd.DataFrame()
df['Job Description'] = data
df['Year'] = jobs['year']
## Since OCI final dataset has no Job Function column, we are merging Function with Industry since they serve the same 
## purpose of added filtering of visualizations. This will make sure OCI and Non OCI skillsets will have the same columns
df['Job Function'] = jobs['Industry'] 

In [80]:
from nltk import pos_tag
from nltk.stem import PorterStemmer

ps = PorterStemmer()


# process the job description.
def prepare_job_desc(desc):
    # tokenize description.
    tokens = word_tokenize(desc)
        
    # Parts of speech (POS) tag tokens.
    token_tag = pos_tag(tokens)
    
    # Only include some of the POS tags.
    include_tags = ['VBN', 'VBD', 'JJ', 'JJS', 'JJR', 'CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens = [tok for tok, tag in token_tag if tag in include_tags]
    
    # stem words.
    stemmed_tokens = [tok.lower() for tok in filtered_tokens]
    return set(stemmed_tokens)

df['job_description_word_set'] = df['Job Description'].map(prepare_job_desc)


skill_keywords1_set = set([ps.stem(tok) for tok in skill_keywords1])# stem the keywords (since the job description is also stemmed.)
skill_keywords1_dict = {ps.stem(tok):tok for tok in skill_keywords1}# use this dictionary to revert the stemmed words back to the original.

degree_keywords1_set = set([ps.stem(tok) for tok in degree_dict.keys()])
degree_keywords1_dict = {ps.stem(tok):tok for tok in degree_dict.keys()}


In [82]:
ind_list = df['Job Function'].unique()
ind_list

array(['Other', 'Financial Services', 'Consulting',
       'Transportation & Logistics', 'Healthcare & Pharmaceuticals',
       'Manufacturing', 'Real Estate & Construction',
       'Professional Services', 'Technology',
       'Government & Non Profit & Education', 'Consumer Packaged Goods',
       'Retail', 'Media & Entertainment & Sports', 'Hospitality',
       'Energy'], dtype=object)

In [87]:
skill_list = []
degree_list = []
skill_year = []
degree_year = []
JobFunction = []
JobFunction_deg = []
tot_postings = []
postings_degree = []

Oci = pd.DataFrame()

for ind in ind_list:
    Oci = df[df['Job Function'] == ind] # Filtering based on Job Function
    for z in range(2011,2021):
        msk = Oci['Year'] == z # Filtering based on year

        num_postings = len(Oci[msk].index)
        for i in range(num_postings):
            job_desc = Oci[msk].iloc[i]['Job Description'].lower()
            job_desc_set = Oci[msk].iloc[i]['job_description_word_set']

            # check if the keywords are in the job description. Look for exact match by token.
            skill_words = skill_keywords1_set.intersection(job_desc_set)
            degree_words = degree_keywords1_set.intersection(job_desc_set)

            # check if longer keywords (more than one word) are in the job description. Match by substring.
            k = 0
            for skill_keyword2 in skill_keywords2:
                # skill keywords.
                if skill_keyword2 in job_desc:
                    skill_list.append(skill_keyword2)
                    k += 1

            # search for the minimum education.
            min_education_level = 999
            for degree_word in degree_words:
                level = degree_dict[degree_keywords1_dict[degree_word]]
                min_education_level = min(min_education_level, level)

            for degree_keyword2 in degree_keywords2:
                # longer keywords. Match by substring.
                if degree_keyword2 in job_desc:
                    level = degree_dict2[degree_keyword2]
                    min_education_level = min(min_education_level, level)

            # If none of the keywords were found, but the word degree is present, then assume it's a bachelors level.
            if min_education_level > 500:
                if 'degree' in job_desc:
                    min_education_level = 1

            skill_list += list(skill_words)
            degree_list.append(min_education_level)
        A = len(skill_list)
        B = len(skill_year)
        C = len(degree_list)
        D = len(degree_year)
        skill_year.extend([z] * (A-B))
        degree_year.extend([z] * (C-D))
        tot_postings.extend([num_postings] * (A-B))
        postings_degree.extend([num_postings] * (C-D))
    A = len(skill_year)
    B = len(JobFunction)
    C = len(degree_year)
    D = len(JobFunction_deg)
    JobFunction.extend([ind] * (A-B))
    JobFunction_deg.extend([ind] * (C-D))

In [90]:
# create the list of skills/knowledge.
df_skills = pd.DataFrame(data={'cnt': skill_list, 
                               'Year': skill_year, 
                               'Job Function': JobFunction, 
                               'Postingcount': tot_postings})
df_skills = df_skills.replace(skill_keywords1_dict)

# group some of the categories together.

msk = df_skills['cnt'] == 'ms excel'
df_skills.loc[msk, 'cnt'] = 'excel'

msk = df_skills['cnt'] == 'excel skills'
df_skills.loc[msk, 'cnt'] = 'excel'

msk = df_skills['cnt'] == 'strong excel'
df_skills.loc[msk, 'cnt'] = 'excel'

msk = df_skills['cnt'] == 'power bi'
df_skills.loc[msk, 'cnt'] = 'powerbi'

msk = df_skills['cnt'] == 'erp system'
df_skills.loc[msk, 'cnt'] = 'erp'

msk = df_skills['cnt'] == 'sql server'
df_skills.loc[msk, 'cnt'] = 'sql'

msk = df_skills['cnt'] == 'sqlexperience'
df_skills.loc[msk, 'cnt'] = 'sql'

msk = df_skills['cnt'] == 'sta'
df_skills.loc[msk, 'cnt'] = 'stata'

msk = df_skills['cnt'] == 'microsoft excel'
df_skills.loc[msk, 'cnt'] = 'excel'

msk = df_skills['cnt'] == 'ms powerpoint'
df_skills.loc[msk, 'cnt'] = 'powerpoint'

msk = df_skills['cnt'] == 'power point'
df_skills.loc[msk, 'cnt'] = 'powerpoint'

msk = df_skills['cnt'] == 'ms power point'
df_skills.loc[msk, 'cnt'] = 'powerpoint'

msk = df_skills['cnt'] == 'microsoft power point'
df_skills.loc[msk, 'cnt'] = 'powerpoint'

msk = df_skills['cnt'] == 'microsoft powerpoint'
df_skills.loc[msk, 'cnt'] = 'powerpoint'

msk = df_skills['cnt'] == 'six sigma'
df_skills.loc[msk, 'cnt'] = 'lean six sigma'

msk = df_skills['cnt'] == 'nlp'
df_skills.loc[msk, 'cnt'] = 'natural language processing'

msk = df_skills['cnt'] == 'convolutional'
df_skills.loc[msk, 'cnt'] = 'convolutional neural network'

msk = df_skills['cnt'] == 'cnn'
df_skills.loc[msk, 'cnt'] = 'convolutional neural network'

msk = df_skills['cnt'] == 'recurrent'
df_skills.loc[msk, 'cnt'] = 'recurrent neural network'

msk = df_skills['cnt'] == 'rnn'
df_skills.loc[msk, 'cnt'] = 'recurrent neural network'

msk = df_skills['cnt'] == 'knn'
df_skills.loc[msk, 'cnt'] = 'nearest neighbors'

msk = df_skills['cnt'] == 'svm'
df_skills.loc[msk, 'cnt'] = 'support vector machine'

msk = df_skills['cnt'] == 'machine vision'
df_skills.loc[msk, 'cnt'] = 'computer vision'

msk = df_skills['cnt'] == 'ab testing'
df_skills.loc[msk, 'cnt'] = 'a/b testing'

msk = df_skills['cnt'] == 'h20'
df_skills.loc[msk, 'cnt'] = 'h2o'

msk = df_skills['cnt'] == 'aws'
df_skills.loc[msk, 'cnt'] = 'amazon web services'

msk = df_skills['cnt'] == 'gcp'
df_skills.loc[msk, 'cnt'] = 'google cloud'

msk = df_skills['cnt'] == 'github'
df_skills.loc[msk, 'cnt'] = 'git'

msk = df_skills['cnt'] == 'postgressql'
df_skills.loc[msk, 'cnt'] = 'postgres'

msk = df_skills['cnt'] == 'tensor'
df_skills.loc[msk, 'cnt'] = 'tensorflow'

msk = df_skills['cnt'] == 'abi'
df_skills.loc[msk, 'cnt'] = 'application binary interface'

In [91]:
df_skillcount = pd.DataFrame()
df_skills_top = pd.DataFrame()
for ind in ind_list:
    df_skillcount_f = df_skills[df_skills['Job Function'] == ind]
    for k in range(2011, 2021):
        if k in df_skillcount_f['Year'].values:
            value = df_skillcount_f[df_skillcount_f['Year'] == k]['Postingcount'].iloc[0]
            df_skillcount = df_skillcount_f[df_skillcount_f['Year'] == k]['cnt'].value_counts().reset_index().rename(columns={'index': 'skill'}).iloc[:30]
            df_skillcount['Year'] = k
            df_skillcount['cnt'] = round((df_skillcount['cnt']/value) * 100)
            df_skillcount['Job Function'] = ind
            df_skills_top = df_skills_top.append(df_skillcount)

In [92]:
df_skills_top = df_skills_top.sort_values(['Job Function', 'Year', 'cnt'], ascending=[True, True, False])

In [93]:
# Visualizing Skills
df_skills_top["Year"] = df_skills_top["Year"].astype(str)

import plotly.express as px

fig = px.bar(df_skills_top, x="skill", y="cnt", animation_frame ='Year', barmode = 'group')
fig.update_layout(barmode='group', xaxis_tickangle=45)
fig.show()

In [94]:
df_skills_top['On Campus'] = 'OCI'

In [95]:
# create the list of degree.
df_degrees = pd.DataFrame(data={'cnt': degree_list, 
                                'Year': degree_year, 
                                'Job Function': JobFunction_deg, 
                                'Postingcount': postings_degree})
df_degrees['degree_type'] = ''


msk = df_degrees['cnt'] == 1
df_degrees.loc[msk, 'degree_type'] = 'bachelors'

msk = df_degrees['cnt'] == 2
df_degrees.loc[msk, 'degree_type'] = 'masters'

msk = df_degrees['cnt'] == 3
df_degrees.loc[msk, 'degree_type'] = 'phd'

msk = df_degrees['cnt'] == 4
df_degrees.loc[msk, 'degree_type'] = 'postdoc'

msk = df_degrees['cnt'] == 2.5
df_degrees.loc[msk, 'degree_type'] = 'mba'

msk = df_degrees['cnt'] > 500
df_degrees.loc[msk, 'degree_type'] = 'not specified'


df_degree_cnt = df_degrees['degree_type'].value_counts().reset_index().rename(columns={'index': 'degree'}).iloc[:50]


In [96]:
DegreeData = pd.DataFrame()
for ind in ind_list:
    df_degreecount = df_degrees[df_degrees['Job Function'] == ind]
    for k in range(2011, 2021):
        if k in df_degreecount['Year'].values:
            value = df_degreecount[df_degreecount['Year'] == k]['Postingcount'].iloc[0]
            df_MinDegree = df_degreecount[df_degreecount['Year'] == k]['degree_type'].value_counts().reset_index().rename(columns={'index': 'Degree'}).iloc[:30]
            df_MinDegree['Year'] = k
            df_MinDegree['degree_type'] = round((df_MinDegree['degree_type']/value) * 100)
            df_MinDegree['Job Function'] = ind
            DegreeData = DegreeData.append(df_MinDegree)

In [97]:
# visualize the degrees.

DegreeData["Year"] = DegreeData["Year"].astype(str)

import plotly.express as px

fig = px.bar(DegreeData, x="Degree", y="degree_type",color = 'Job Function', animation_frame ='Year', barmode = 'group')
fig.update_layout(barmode='group', xaxis_tickangle=45)
fig.show()

In [98]:
DegreeData['On Campus'] = 'OCI'

In [99]:
### Dataset Creation for Tableau ###

In [100]:
### Select NonOciSkills.csv file from your working directory ###

root = Tk()
root.filename =  filedialog.askopenfilename()
print (root.filename)
root.destroy()

C:/Users/krish/Desktop/UMD/Capstone/skill_visual.csv


In [101]:
df1 = pd.read_csv(root.filename)

In [102]:
df2 = df1.append(df_skills_top)

In [103]:
df2 = df2.reset_index(drop = True)

In [104]:
df2.to_csv("TopSkills.csv")

In [105]:
### Select NonOciDegree.csv file from your working directory ###

root = Tk()
root.filename =  filedialog.askopenfilename()
print (root.filename)
root.destroy()

C:/Users/krish/Desktop/UMD/Capstone/JLMinDegree.csv


In [106]:
df3 = pd.read_csv(root.filename)

In [107]:
df4 = df3.append(DegreeData)

In [108]:
df4 = df4.reset_index(drop = True)

In [109]:
df4.to_csv("MinDegree.csv")

In [110]:
#### The End #####